In [ ]:
# 匯入套件


# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome()

# 螢幕最大化
#driver.maximize_window()

# 放置爬取的資料
listBook = []
urlList = []


# 以 function 名稱，作為爬蟲流程


# 走訪頁面
def visit():
    driver.get('https://www.bookwormzz.com/zh/')

# 收集小說書本目錄url
def search():
    
    sleep(2)
    #按下
    books = driver.find_elements(By.CSS_SELECTOR, "a.ui-collapsible-heading-toggle.ui-btn.ui-btn-icon-left.ui-btn-a.ui-icon-plus")
    for book in books:
        sleep(2)
        book.click()
    
    bookurls = driver.find_elements(By.CSS_SELECTOR, "a.ui-btn.ui-btn-icon-right.ui-icon-carat-r")
    for bookurl in bookurls:
        listBook.append(bookurl.get_attribute('href'))
    
    #print(listBook)
    


def findPage():
    # 等待篩選元素出現
    # 等5秒
    for bookpage in listBook:
        try:
            driver.get(bookpage)
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located( 
                    (By.CSS_SELECTOR, "a.ui-link.ui-btn.ui-icon-bullets.ui-btn-icon-left")   
                )
            )

            #按下篩選元素，使項目浮現
            driver.find_element(By.CSS_SELECTOR, "a.ui-link.ui-btn.ui-icon-bullets.ui-btn-icon-left").click()

            # 等待一下
            sleep(2)

            # 擷取每個頁面的url
            aList = driver.find_elements(By.CSS_SELECTOR, "#book_toc > div.ui-content a")


            for a in aList:
                aTitle = a.text
                print(aTitle)

                aLink = a.get_attribute('href')
                print(aLink)

                urlList.append({
                    "Title": aTitle,
                    "Link": aLink
                })
        except:
            print("出現錯誤")
            print(bookpage)
        


# 將 list 存成 json
def saveJson():
    fp = open("novel.json", "w", encoding='utf-8')
    fp.write( json.dumps(urlList, ensure_ascii=False) )
    fp.close()
    
# 關閉瀏覽器
def close():
    driver.quit()
# 下載網頁的html檔
def download():
#     # 開啟 json 檔案
#     fp = open("novel.json", "r", encoding='utf-8')
    
#     #取得 json 字串
#     strJson = fp.read()
    
#     # 關閉檔案
#     fp.close()
    
#     # 將 json 轉成 list (裡面是 dict 集合)
#     listResult = json.loads(strJson)
    
#     # 下載網頁
#     for index, obj in enumerate(listResult): 
#         if index <= 2:
#             os.system("curl {} -o {}.html".format(obj["Link"], obj["Title"]))

# 下載網頁內容(txt檔)
def downloadtxt():
    fp = open("novel.json", "r", encoding='utf-8')
    
    #取得 json 字串
    strJson = fp.read()
    
    # 關閉檔案
    fp.close()
    
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)
    for index, obj in enumerate(listResult):
        try:
            driver.get(obj["Link"])
            sleep(2)
            article = driver.find_element(By.CSS_SELECTOR, "#html > div:nth-child(1)").text
            fp = open("{}.txt".format(obj["Title"]), "w", encoding='utf-8')
            fp.write( article )
            fp.close()
        except:
            print("出現錯誤")
            print(obj["Title"])
            print(obj["Link"])
    


# 主程式
visit()
search()
findPage()
saveJson()
close()
# download()
downloadtxt() 